# Lab 7: One-Sample t-Tests, Confidence Intervals, and Cohen's d in Behavioral Health Data

**Learning Objectives**  
By the end of this lab, you will be able to:  
1. **Formulate and test hypotheses for one mean** using one-sample *t*-tests (and *z*-tests conceptually) in the context of public policy and health data.  
2. **Calculate and interpret confidence intervals** for a sample mean, and understand the relationship between a 95% CI and a two-sided hypothesis test at α = 0.05.  
3. **Quantify effect size** using **Cohen's d**, and discuss why statistical significance (p-value) does not always imply practical significance (considering confidence intervals and effect sizes).

## Important Note About the Data

**This lab uses synthetic data designed to illustrate statistical concepts and common patterns in policy research.** While the data are based on real-world policy research findings and demonstrate authentic challenges (selection bias, confounding, statistical vs. practical significance), exact effect sizes may differ from published studies. 

For actual policy evaluation, researchers use more sophisticated causal inference designs (difference-in-differences, regression discontinuity, instrumental variables, randomized controlled trials) to establish causality beyond simple group comparisons. The patterns you'll observe here—especially regarding selection bias in policy adoption—reflect real methodological challenges in observational research.

## Background & Variables

We will use a synthetic dataset (~11,600 adolescents at baseline, with follow-up data at age ~21) derived from the ABCD study. This dataset includes state-level policy indicators and individual health/behavior outcomes. Key variables for this lab include:

- **Marijuana Law Classification** (`le_l_lawsmj__mjlaws__state_indicator`): Categorical state policy for marijuana access (1 = Recreational legal, 2 = Medical only, 3 = CBD-only/Low THC, 4 = No legal access).  
- **ACA Medicaid Expansion** (`le_l_aca__addr1__acaexpand_indicator`): Whether the participant's state expanded Medicaid under the Affordable Care Act (1 = Yes/Expanded, 0 = No).  
- **Opioid Prescribing Rate** (`le_l_rxopioid__addr1__opioidrx_rate__yb0`): Annual opioid prescription rate in the state (per 100 population, baseline year; national average ≈ 55). Higher values = more opioid prescriptions per capita.  
- **PDMP "Must Access" Law** (`le_l_rxmonit__addr1__pdmp__mustaccess_indicator`): Indicator if the state had a Prescription Drug Monitoring Program must-check mandate for providers (1 = Yes, 0 = No). This policy aims to curb excessive prescribing.  

*Note:* Some variables use special codes for missing data (e.g., `999` for questionnaire non-response). We will filter out such values in our analyses as needed so they do not distort our results.

## Setup

Run the setup cell below to import libraries and load the dataset for Lab 7. The dataset is the same one used in Lab 6 (with both baseline and follow-up data). We’ll use pandas for data handling, SciPy for statistical tests, and matplotlib for plotting. A helper function `savefig()` is provided to save figures to a `figures/` directory.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Display all columns
pd.set_option('display.max_columns', None)

# Load the dataset (make sure the CSV is in the same directory)
df = pd.read_csv('data/L6L10/ABCD_synthetic.csv')

# Quick check of data shape
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

# Set up figure directory and helper function
from pathlib import Path
FIG_DIR = Path('figures')
FIG_DIR.mkdir(exist_ok=True)
def savefig(name, dpi=150):
    """Save current Matplotlib figure to figures/<name>.png"""
    plt.tight_layout()
    if not name.endswith('.png'):
        name += '.png'
    plt.savefig(FIG_DIR / name, dpi=dpi)
    print(f"Figure saved as {name}")


### 🧭 Mini Lesson: Cohen’s *d* (one-sample) as effect size, beyond *p*-values

<p align="center">
  <img src="cohensdviz.png" alt="Two panels showing bell-curve overlap: small effect (high overlap) vs large effect (low overlap)" width="720">
</p>


**What it is.**  
A standardized mean difference: how far $ \bar{x} $ is from a benchmark $ \mu_0 $ in **standard deviation units**.  
Cohen’s _d_ is calculated as $ d = \frac{\bar{x} - \mu_0}{s} $.

- **Benchmarks (rules of thumb):** $d \approx 0.2$ small, $0.5$ medium, $0.8+$ large. Context matters.  
- **Why it’s here:** *p*-values tell you “rare vs not” under $H_0$; *d* tells you **how big** the effect is.  
- **Assumptions:** same as the one-sample *t*: independent observations, sensible to summarize by a mean, and not dominated by outliers.

---

**Report checklist (use this order in your reflection):**  
$\bar{x}$ (with units), 95% CI, *p*-value, and **$d$** with a plain-language label (small / medium / large).

---

**In this lab:**
- **Activity 1:** Compute CI + two-sided test; add $d$.  
- **Activity 2:** One-sided test; always report $d$ and discuss **practical vs statistical** significance.  
- **Activity 3:** Same mechanics; pre-specify direction, don’t choose tails after peeking.

---

**Tip:**  
Cohen’s *d* complements statistical significance.  
Even if *p* < 0.05, a small $d$ means the effect—though “real”—might be trivial in practice.


## Activity 1. Cannabis Policy and Youth Marijuana Use

**Demo:** Does legalizing recreational marijuana affect youth cannabis use? In this demo, we focus on youths from states with **Recreational marijuana laws** and test whether their average cannabis use is different from youths in states with **no legal access**. Our outcome of interest will be the **total marijuana dose (TLFB)** the youth has ever used by early adulthood (age ~21). We’ll conduct a one-sample t-test to see if the mean total dose in recreational-marijuana states differs from a reference value. Here, we’ll use the mean in "no legal access" states as the reference (as if it were a known population value).

*Steps:*  
1. **State the hypotheses:** We want to test if the mean total marijuana dose (TLFB) in recreational states is different from that in no-access states. Let μ = mean total dose used in rec-legal states. Our null hypothesis is H<sub>0</sub>: μ = μ<sub>0</sub>, where μ<sub>0</sub> is the no-law states’ average. The two-sided alternative H<sub>a</sub>: μ ≠ μ<sub>0</sub>. (We suspect it might be higher, but we’ll test two-sided to be conservative.)  
2. **Compute sample statistics:** Calculate the sample mean, standard deviation, and standard error for total marijuana dose (TLFB) in the recreational-law group. We’ll use wave 21 (age ~21) data for a cumulative measure of use. We will treat individuals with no use (NaN in the data) as 0 to include them in the average.  
3. **One-sample t-test:** Compute the t-statistic: t = (mean - μ<sub>0</sub>) / SE. Determine degrees of freedom (df = n - 1). Calculate the p-value for the two-sided test.  
4. **95% Confidence Interval:** Compute the 95% CI for the true mean μ. This is mean ± t·SE, where t is the critical value for df at 95% confidence (~1.96 for large df).  
5. **Effect size (Cohen’s d):** Calculate d = (mean - μ<sub>0</sub>) / s, which measures the difference in means in standard deviation units. Interpret this magnitude (small ~0.2, medium ~0.5, large ~0.8).  
6. **Interpret results:** We will examine the p-value and CI to decide if the null hypothesis can be rejected, and consider whether the difference is practically meaningful.

Let's walk through these steps in code. We’ll first determine μ<sub>0</sub> using the no-access states, then perform the analysis for recreational states.


In [ ]:
# 1. Filter the data for follow-up (age ~21) and the relevant groups
followup = df[df['wave'] == 21].copy()

# Define policy groups: rec-legal (1) vs no-legal-access (4)
rec_states = followup[followup['le_l_lawsmj__mjlaws__state_indicator'] == 1]
no_states = followup[followup['le_l_lawsmj__mjlaws__state_indicator'] == 4]

# Define the outcome variable for total marijuana dose (TLFB)
outcome_var = 'su_y_tlfb__mj_totdose'

# 2. Compute the reference mean (mu0) from no-legal states (treat NaN as 0 for never-users)
dose_no = no_states[outcome_var].fillna(0)  # fill NaN with 0
mu0 = dose_no.mean()
print(f"Mean total marijuana dose (TLFB) in no-law states (mu0) = {mu0:.3f}")

# Get data for rec-legal states (fill NaN with 0)
dose_rec = rec_states[outcome_var].fillna(0)
n = dose_rec.count()  # count non-NaN after fill (which is full count)
mean_rec = dose_rec.mean()
sd_rec = dose_rec.std(ddof=1)
se_rec = sd_rec / np.sqrt(n)
print(f"Rec states: mean = {mean_rec:.3f}, SD = {sd_rec:.3f}, n = {n}")

# 3. One-sample t-test for H0: mu = mu0
dfree = n - 1
t_stat = (mean_rec - mu0) / se_rec
# Two-sided p-value:
p_val = stats.t.sf(abs(t_stat), dfree) * 2

# 4. 95% confidence interval for the mean
t_crit = stats.t.ppf(0.975, dfree)  # two-tailed 95% critical t
ci_low = mean_rec - t_crit * se_rec
ci_high = mean_rec + t_crit * se_rec

# 5. Cohen's d effect size
d = (mean_rec - mu0) / sd_rec

# Print the results
print(f"t({dfree}) = {t_stat:.3f}, p-value = {p_val:.3g}")
print(f"95% CI for mean = [{ci_low:.2f}, {ci_high:.2f}] doses")
print(f"Cohen's d = {d:.3f}")


Running the above, we get the summary statistics and test results for the recreational states group. Now, let's visualize the data distribution and reference point:


In [ ]:
# Plot a histogram of total marijuana dose (TLFB) in rec-legal states
plt.figure(figsize=(6,4))
plt.hist(dose_rec, bins=30, color='skyblue', edgecolor='white')
plt.axvline(mu0, color='red', linestyle='--', linewidth=2, label=f"No-law mean = {mu0:.1f}")
plt.axvline(mean_rec, color='green', linestyle='-', linewidth=2, label=f"Rec-law mean = {mean_rec:.1f}")
plt.xlabel('Total marijuana dose (TLFB) by age 21')
plt.ylabel('Number of youth')
plt.title('Distribution of Total Marijuana Dose (TLFB) in Rec-Legal States')
plt.legend()
savefig('01_demo_rec_vs_nolaw_tlfb')
plt.show()

**Observations:** The recreational-law states have a sample mean of ≈52.39 total marijuana doses (TLFB) by age 21, compared to ≈53.77 doses in no-access states. The two-sided one-sample t-test (benchmark μ₀ = no-law mean) yields t(2114) = -4.450, p = 9.05×10⁻⁶. We reject H₀ and conclude the rec-legal mean differs from the no-law mean and, in these data, is slightly lower. The 95% confidence interval for the rec-legal mean is [51.78, 53.00] doses, which lies below the no-law mean (53.77).

Visual check: In the histogram, the rec-law mean (green solid line) sits to the left of the no-law reference (red dashed), with most of the distribution centered slightly below the red line; the spike at 0 reflects never-users coded as 0. This visual pattern matches the statistical result.

However, Cohen’s d = -0.097 (small). The difference is about a tenth of a standard deviation and likely of limited practical significance. With a large sample, even small effects can be statistically significant—interpret effect size and CI alongside the p-value.

### Your Turn 1: Testing Marijuana Dose in Recreational States

**Context:** In Activity 1, we compared recreational-law states to no-law states and found significantly higher marijuana use in recreational states. Now it's your turn to apply these same methods to test a specific hypothesis about marijuana dose.

**Your Task:** Test whether youths in **recreational marijuana states** have marijuana use that differs from the average in **no-law states** at age ~21 (wave 21). Use the **total marijuana dose (TLFB)** variable (`su_y_tlfb__mj_totdose`).

**Steps to Complete:**
1. Filter the data to wave 21 (follow-up, age ~21)
2. Compute the **reference mean (μ₀)** from no-law states (marijuana law indicator = 4)
3. Extract data for **recreational states** (marijuana law indicator = 1)
4. Perform a **one-sample t-test** comparing recreational state mean to μ₀
5. Calculate the **95% confidence interval** for the mean in recreational states
6. Calculate **Cohen's d** effect size
7. Create a **histogram visualization** showing the distribution with reference lines

**Important Notes:**
- The marijuana dose variable has `NaN` values for people who never used marijuana. Treat these as **0** (never-users) by using `.fillna(0)`.
- Use `ddof=1` when calculating standard deviation (sample SD, not population).
- For the t-test, test the **two-sided alternative** hypothesis (H₀: μ = μ₀, Hₐ: μ ≠ μ₀).
- Your histogram should show both the reference mean (no-law states) and the sample mean (recreational states) as vertical lines.

---

### Getting Started with Copilot

Use these prompts sequentially to build your code. Copy each prompt into Copilot and review the suggested code before moving to the next step.

**Prompt 1:** Filter the dataframe `df` to wave 21 and create two groups: `rec_states` where marijuana law indicator equals 1, and `no_states` where it equals 4. Store the marijuana dose variable name as `outcome_var`.

**Prompt 2:** For the `no_states` group, extract the outcome variable, fill NaN values with 0, and compute the mean as `mu0`. Print the result with a descriptive message.

**Prompt 3:** For the `rec_states` group, extract the outcome variable, fill NaN values with 0, and compute the sample size `n`, mean `mean_rec`, standard deviation `sd_rec` (with ddof=1), and standard error `se_rec`. Print these statistics.

**Prompt 4:** Perform a one-sample t-test comparing `mean_rec` to `mu0`. Calculate degrees of freedom, t-statistic, and two-sided p-value using `stats.t.sf()`. Also compute the 95% confidence interval using the critical t-value from `stats.t.ppf(0.975, dfree)`.

**Prompt 5:** Calculate Cohen's d effect size as the difference between `mean_rec` and `mu0` divided by `sd_rec`. Print all test results: t-statistic, p-value, 95% CI, and Cohen's d.

**Prompt 6:** Create a histogram of marijuana dose in recreational states. Add vertical lines for `mu0` (red dashed) and `mean_rec` (green solid) with labels. Include axis labels, title, and legend. Use `savefig('yt1_rec_vs_nolaw')` to save.

**Prompt 7:** Display the plot with `plt.show()`.


In [ ]:
=== Your Turn 1: Testing Marijuana Dose in Recreational States ===

# TODO: Filter data to wave 21 and create groups
# followup = ..., rec_states = ..., no_states = ...
# outcome_var = 'su_y_tlfb__mj_totdose'

# TODO: Compute reference mean (mu0) from no-law states
# Remember to use .fillna(0) for never-users

# TODO: Get recreational states data and compute statistics
# n, mean_rec, sd_rec (ddof=1), se_rec

# TODO: Perform one-sample t-test (two-sided)
# dfree, t_stat, p_val using stats.t.sf()

# TODO: Calculate 95% confidence interval
# t_crit from stats.t.ppf(0.975, dfree)

# TODO: Calculate Cohen's d effect size

# TODO: Print results (t-stat, p-value, CI, Cohen's d)

# TODO: Create histogram visualization
# hist(dose_rec), axvline for mu0 and mean_rec

# TODO: Save and show figure

# your code here

## 💡 Mini Lesson: Practical Effect Size

**Why:**  
A small p-value only tells us that a difference is unlikely due to chance—it doesn’t say whether the difference *matters*.  
**Effect size** quantifies *how big* that difference is, in real and standardized terms.

---

### Two Ways to Describe Effect Size

**1. Raw difference (Δmean):**  
\[
\Delta = \bar{x} - \mu_0
\]  
Keep the original units (e.g., prescriptions per 1,000 residents).

**2. Standardized difference (Cohen’s d, one-sample):**  
\[
d = \frac{\bar{x} - \mu_0}{s}
\]  
where \( s \) is the sample standard deviation.  
(Optional small-sample correction → Hedges’ g:  
\[
g = d \times \left(1 - \frac{3}{4n - 9}\right)
\])

---

### Interpreting *d* (rule of thumb, not law)

| Magnitude | Interpretation |
|------------|----------------|
| ~0.2 | Small |
| ~0.5 | Medium |
| ~0.8 | Large |

Always interpret in context—some small effects can matter (e.g., policy or health outcomes).

---

### Reporting Template

> Compared to μ₀ = [value units], the sample mean was  
> x̄ = [value], Δmean = [value units],  
> 95% CI = [L, U units], Cohen’s d = [value] (Hedges’ g = [value, if used]).  
> Interpretation: [one concise sentence on meaningfulness in context].

---

### Common Pitfalls

- **Significant p with tiny d ≠ meaningful** difference.  
- **Non-significant p with moderate d** may suggest more data are needed (low power).  
- Always report **units, direction, and uncertainty (CI)** alongside p and d.

---

**Next:** Apply these ideas in Activity 2 by calculating Δmean, 95% CI, and Cohen’s d for your sample.


## Activity 2. Health Policy and Social Determinants of Health

**Demo:** Do public health policies translate into better community conditions for children and families? Here we examine states' **Medicaid expansion** status under the ACA (Affordable Care Act) and a measure of social determinants: the **Child Opportunity Index (COI)**. The COI is a z-score (nationally normed) that captures neighborhood resources like quality schools, green spaces, economic opportunity, and health resources. States that expanded Medicaid aimed to increase healthcare access and improve overall well-being, which might be reflected in higher COI scores. We will test whether youths in expansion states have, on average, higher COI scores than the national average. Since the COI is a z-score, the national mean is **0**. Let's test if the mean COI in expansion states is **greater than 0** (if the policy had a beneficial effect on community conditions). This will be a **one-sided** test.

*Steps:*  
1. **Hypotheses:** Let μ = mean COI z-score in expansion-state youth. H<sub>0</sub>: μ = 0 (no difference from national average), H<sub>a</sub>: μ > 0 (mean is above the national average, indicating better neighborhood opportunity). We choose a one-tailed test since we have a directional hypothesis (we expect higher COI in expansion states).  
2. **Sample statistics:** Filter the baseline data (age ~16) for participants in expansion states (where `acaexpand_indicator = 1`). Calculate mean, SD, SE of COI z-score.  
3. **One-sample t-test:** Compute t = (mean - 0)/SE. Here a positive t (mean > 0) in the right tail would support H<sub>a</sub>. Compute the one-sided p-value for t (upper tail).  
4. **95% CI:** Calculate the two-sided 95% confidence interval for the mean. Even though our test is one-sided, we will compute the 95% CI to see the plausible range for the mean COI z-score.  
5. **Cohen's d:** Compute d = (mean - 0)/SD, which indicates how many standard deviations above (or below) the national average the expansion states are.  
6. **Interpretation:** Determine if we can reject H<sub>0</sub>. Consider the size of the effect in practical terms: is the increase in neighborhood opportunity modest or substantial?

Let's perform the analysis:

In [ ]:
# 1. Filter baseline data for ACA expansion states
baseline = df[df['wave'] == 16].copy()
expansion_group = baseline[baseline['le_l_aca__addr1__acaexpand_indicator'] == 1]

# Define outcome variable for Child Opportunity Index (COI z-score)
coi_var = 'le_l_coi__addr1__coi__total__national_zscore'

# Extract COI data for expansion states, filter missing codes
coi_exp = expansion_group[coi_var].dropna()
coi_exp = coi_exp[coi_exp != 999]  # remove any coded missing values
n = len(coi_exp)
mean_coi = coi_exp.mean()
sd_coi = coi_exp.std(ddof=1)
se_coi = sd_coi / np.sqrt(n)
print(f"Expansion states: mean COI z-score = {mean_coi:.3f}, SD = {sd_coi:.3f}, n = {n}")

# 2. Hypothesized value and one-sample t-test (one-sided H_a: mu > 0)
mu0 = 0.0  # national average for z-score
dfree = n - 1
t_stat = (mean_coi - mu0) / se_coi
p_val = stats.t.sf(t_stat, dfree)   # upper-tail p (because H_a is mu > 0)

# 3. 95% CI for mean COI z-score
t_crit = stats.t.ppf(0.975, dfree)
ci_low = mean_coi - t_crit * se_coi
ci_high = mean_coi + t_crit * se_coi

# 4. Cohen's d
d = (mean_coi - mu0) / sd_coi

# Output results
print(f"t({dfree}) = {t_stat:.3f}, one-tailed p = {p_val:.3g} (H_a: mu > 0)")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")
print(f"Cohen's d = {d:.3f}")

Next, we'll visualize the result with a plot. We can show the sample mean with its 95% CI and mark the national average (z-score = 0):

In [ ]:
# Plot mean COI z-score with 95% CI as a bar chart for better visualization
plt.figure(figsize=(5,5))
mean_val = mean_coi
ci_low_val, ci_high_val = ci_low, ci_high
error = ci_high_val - mean_val  # symmetric error for bar chart

# Create bar with error bar
plt.bar(1, mean_val, color='steelblue', alpha=0.7, width=0.4, label='Expansion states')
plt.errorbar(1, mean_val, yerr=error, fmt='none', color='black', capsize=10, capthick=2, linewidth=2)

# Add national average reference line
plt.axhline(mu0, color='red', linestyle='--', linewidth=2, label='National avg (z = 0)')

# Add text annotation for mean value
plt.text(1, mean_val + 0.05, f'Mean = {mean_val:.3f}\n95% CI [{ci_low_val:.3f}, {ci_high_val:.3f}]', 
         ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.xlim(0.5, 1.5)
plt.ylim(min(ci_low_val - 0.05, -0.05), max(ci_high_val + 0.1, 0.15))
plt.xticks([1], ['ACA Expansion\nStates'])
plt.ylabel('Child Opportunity Index (z-score)')
plt.title('Mean COI in ACA Expansion States (baseline)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
savefig('02_demo_expansion_coi')
plt.show()

**Observations:** The average COI z-score in expansion-state youth is **0.022** (SD = 0.030, n = 6,849), with a 95% CI of **[0.021, 0.023]**. This is statistically significantly above the national average of 0. The one-sample t-test yields **t(6848) = 59.502, p ≈ 0** (essentially zero), so we **reject H<sub>0</sub>** and conclude that youths in Medicaid expansion states live in neighborhoods with significantly higher opportunity than the national average.

However, let's examine the **magnitude** of this difference. The effect size **Cohen's d = 0.719** is considered a **medium-to-large effect**, indicating that expansion states are about 0.7 standard deviations above the national average. This is a meaningful difference in z-score terms. Yet, the **raw difference** in COI z-score is only **0.022** — a very small absolute shift on the z-score scale. The 95% confidence interval **[0.021, 0.023]** is extremely narrow (does not include 0, confirming significance), but the range is also very small.

**Interpretation:** This demonstrates a key lesson about **statistical vs. practical significance**. With a **very large sample** (n = 6,849), even a **tiny difference** (0.022 z-score units) becomes highly statistically significant (p ≈ 0). The Cohen's d suggests a moderate-to-large effect because the standard deviation is also small (0.030), magnifying the relative size of the difference. 

In practical terms: Medicaid expansion states do have slightly better neighborhood conditions on average, but the absolute improvement is **modest** — about 2% of a standard deviation in raw z-score units. This could reflect:
1. **Pre-existing differences**: States that expanded Medicaid may have already had slightly better baseline conditions (selection effect).
2. **Indirect effects**: Medicaid expansion may have modest downstream effects on community investment and resources.
3. **Small signal**: The policy effect on neighborhood-level COI may be real but small in magnitude.

**Key takeaway:** A tiny p-value and a large sample don't guarantee a large real-world effect. Always examine the **effect size (Cohen's d)**, the **confidence interval**, and the **absolute magnitude** of the difference to assess practical importance. Here, the result is statistically robust but practically modest.

### Your Turn 2: Does Medicaid Expansion Improve Neighborhood Conditions for Low-Income Families?

**Context:** In Activity 2, we found that youths in Medicaid expansion states have slightly higher COI z-scores than the national average. But this doesn't tell us if the **policy itself** caused the improvement, or if expansion states simply had better conditions to begin with (selection bias).

**The Problem:** Comparing expansion states to the national average conflates **state differences** with **policy effects**. Richer states may have expanded Medicaid *and* have better neighborhoods for unrelated reasons.

**Better Design:** Focus on **low-income families only** (the target population for Medicaid expansion) and compare:
- **Expansion states**: Low-income families in states that expanded Medicaid
- **Non-expansion states**: Low-income families in states that did NOT expand Medicaid

By holding income constant, we isolate the policy effect. If Medicaid expansion improves community conditions for eligible families, we should see **higher COI in expansion-state low-income families** compared to non-expansion-state low-income families.

**Your Task:** 
1. Define **low-income families** (income categories 1-5: < $35k/year, roughly Medicaid-eligible)
2. Split this group by expansion status into two groups
3. For **non-expansion low-income**, compute the mean COI as the reference value (μ₀)
4. For **expansion low-income**, run a **one-sample t-test**:
   - H₀: μ = μ₀ (no difference between expansion and non-expansion low-income families)
   - Hₐ: μ > μ₀ (expansion-state low-income families have better neighborhoods)
5. Calculate **pooled Cohen's d** to compare the two groups
6. Create **three visualizations**: side-by-side comparison, effect size, and overlaid distributions

**Important Notes:**
- Income variable: `ab_p_demo__income__hhold_001__v01` (categories 1-5 = <$35k)
- Expansion variable: `le_l_aca__addr1__acaexpand_indicator` (1 = Yes, 0 = No)
- COI variable: `le_l_coi__addr1__coi__total__national_zscore`
- Use baseline data (wave 16)
- Filter out missing values (999) and NaN
- Use **pooled standard deviation** for Cohen's d (combining both groups)
- This is a **one-tailed test** (Hₐ: μ > μ₀)

---

### Getting Started with Copilot

**Part 1: Statistical Analysis (First Code Cell)**

Use these prompts to build your statistical analysis code:

**Prompt 1:** Filter `df` to wave 16 as `baseline`. Create `low_income_all` by filtering baseline to income categories 1-5. Split this into `low_income_expansion` (expansion indicator = 1) and `low_income_no_expansion` (expansion indicator = 0). Define `coi_var` as the COI variable name.

**Prompt 2:** For the non-expansion group, extract COI data, drop NaN, filter out 999 values, and compute the reference mean `mu0` and standard deviation `sd_ref`. Also get sample size `n_ref`. Print summary statistics.

**Prompt 3:** For the expansion group, extract COI data, drop NaN, filter out 999, and compute `n_exp`, `mean_exp`, `sd_exp`, and `se_exp`. Print summary statistics.

**Prompt 4:** Perform a one-sample t-test comparing `mean_exp` to `mu0`. Calculate degrees of freedom, t-statistic, and one-tailed p-value (upper tail) using `stats.t.sf()`. Calculate 95% CI.

**Prompt 5:** Calculate pooled standard deviation using the formula: sqrt(((n_exp-1)*sd_exp^2 + (n_ref-1)*sd_ref^2) / (n_exp + n_ref - 2)). Then compute Cohen's d as (mean_exp - mu0) / pooled_sd.

**Prompt 6:** Print all results including sample sizes, means, t-statistic, p-value, CI, Cohen's d, and interpretation of statistical significance.

---

**Part 2: Visualization (Second Code Cell)**

Use these prompts to build your visualization code:

**Prompt 7:** Create a figure with 2 subplots side-by-side using `plt.subplots(1, 2, figsize=(13, 5))`. In the left panel, create a bar chart comparing non-expansion mean (mu0) and expansion mean (mean_exp) with error bars for the expansion group.

**Prompt 8:** In the right panel, create a horizontal bar showing the effect size (difference between means) with vertical lines marking both means and the national average at 0. Add text annotation showing difference, Cohen's d, and p-value.

**Prompt 9:** Create a third figure with overlaid histograms of COI distributions for both groups (non-expansion in light coral, expansion in medium sea green). Add vertical lines for both means and the national average. Include legend, labels, and title.

**Prompt 10:** Save all figures using `savefig()` and display with `plt.show()`.

In [ ]:
=== Your Turn 2: Statistical Analysis - Low-Income Medicaid Expansion ===

# TODO: Filter to baseline (wave 16) and define low-income families
# baseline = ..., income_var = '...', categories 1-5
# low_income_all = ..., split by expansion status (0 vs 1)

# TODO: Define COI variable
# coi_var = 'le_l_coi__addr1__coi__total__national_zscore'

# TODO: Compute reference mean (mu0) from NON-EXPANSION low-income group
# Extract COI, dropna(), filter != 999
# n_ref, mu0, sd_ref

# TODO: Get EXPANSION low-income group statistics
# Extract COI, dropna(), filter != 999
# n_exp, mean_exp, sd_exp, se_exp

# TODO: One-sample t-test (one-tailed, Ha: mu > mu0)
# df_exp, t_stat, p_val using stats.t.sf()

# TODO: Calculate 95% confidence interval
# t_crit from stats.t.ppf(0.975, df_exp)

# TODO: Calculate pooled Cohen's d
# pooled_sd = sqrt(((n_exp-1)*sd_exp^2 + (n_ref-1)*sd_ref^2) / (n_exp+n_ref-2))
# d = (mean_exp - mu0) / pooled_sd

# TODO: Print comprehensive results
# Include: both group statistics, t-test results, CI, Cohen's d, interpretation

# your code here


In [ ]:
=== Your Turn 2: Visualization - Low-Income Medicaid Expansion ===

# TODO: Create figure with 2 subplots (1 row, 2 columns)
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# TODO: LEFT PANEL - Side-by-side bar chart
# Bar chart with mu0 (non-expansion) and mean_exp (expansion)
# Add error bars for expansion group (CI)
# Add horizontal line at national avg (z=0)
# Add value labels and sample sizes

# TODO: RIGHT PANEL - Effect visualization
# Horizontal bar showing difference (mean_exp - mu0)
# Vertical lines for mu0, mean_exp, and national avg (0)
# Text annotation with difference, Cohen's d, p-value

# TODO: Add titles, labels, legends, grid for both panels
# Save figure: savefig('yt2_expansion_effect_low_income')

# TODO: Create THIRD figure - Overlaid histograms
# plt.figure(figsize=(9, 5))
# Histogram for non-expansion (light coral, alpha=0.6)
# Histogram for expansion (medium sea green, alpha=0.6)
# Vertical lines for both means and national avg
# Labels, title, legend

# TODO: Save and show all figures
# savefig('yt2_expansion_distributions')
# plt.show()

# your code here


**Interpretation Instructions:**

After running the code above, answer the following:

1. **Descriptive Comparison:**
   - What is the mean COI for low-income families in **non-expansion states** (the reference group)?
   - What is the mean COI for low-income families in **expansion states**?
   - What is the **raw difference** between these means?

2. **Statistical Significance:**
   - Is the mean COI in expansion states significantly **higher** than in non-expansion states? (Check the p-value)
   - Does the 95% confidence interval for the expansion-state mean **include** the non-expansion reference value?
   - What does this tell you about whether the difference could be due to chance?

3. **Effect Size and Practical Significance:**
   - What is Cohen's d? Is it small (~0.2), medium (~0.5), or large (~0.8)?
   - In practical terms, how much better (or worse) are neighborhoods for low-income families in expansion states?
   - Is this a meaningful difference in real-world terms, or just statistically detectable?

4. **Policy Implications:**
   - Does this analysis support the hypothesis that **Medicaid expansion improves neighborhood conditions** for its target population (low-income families)?
   - What are alternative explanations for the pattern you observe? (e.g., selection bias: maybe states that expanded already had slightly better conditions even among low-income families?)
   - What would you need to establish **causation** rather than just correlation?

5. **Comparison to Activity 2 Demo:**
   - In Activity 2, we compared expansion states to the **national average** and found a significant effect. How does that compare to this analysis, where we compared low-income expansion families to low-income non-expansion families?
   - Which design (national average vs. non-expansion control group) provides stronger evidence for a **policy effect**? Why?

Write your interpretation in a markdown cell below, citing specific statistics (means, difference, p-value, Cohen's d, CI) from your output.

## Activity 3. Opioid Policy Effects and Practical vs. Statistical Significance

**Demo:** In this demo, we examine a case where a result can be statistically significant but practically small. Consider the impact of a strict Prescription Drug Monitoring Program (PDMP) policy. **“Must access” PDMP laws** require prescribers to check a database before prescribing opioids, intended to reduce excessive opioid prescriptions. We’ll test if states **without** a must-access PDMP have a higher opioid prescribing rate than the national average (which is about 55 per 100 people). 

Specifically, let’s focus on baseline data and take the group of participants in states without a must-access PDMP. We test H<sub>0</sub>: μ = 55 vs H<sub>a</sub>: μ > 55 (one-sided, expecting higher prescribing in absence of strict monitoring). Our outcome is the state opioid prescription rate (`opioidrx_rate`) which is the same for everyone in a state (but appears in each individual’s data).

*Steps:*  
1. **Hypotheses:** μ = mean opioid prescribing rate in non-must-access states. H<sub>0</sub>: μ = 55, H<sub>a</sub>: μ > 55 (we anticipate these states might have above-average prescribing due to laxer policy).  
2. **Sample statistics:** Filter baseline (wave == 16) for participants in states where `pdmp_mustaccess_indicator = 0` (no mandatory PDMP). Compute mean, SD, SE of the prescribing rate. (Note: Many individuals share the same state values, but we’ll treat the sample at face value.)  
3. **One-sample t-test:** Calculate t = (mean - 55)/SE, with a one-sided test (upper tail).  
4. **95% CI:** Compute the 95% CI for the mean.  
5. **Cohen’s d:** Compute d = (mean - 55)/SD.  
6. **Interpretation:** We expect to reject H<sub>0</sub> if the mean is even slightly above 55 due to the large sample. Pay special attention to the effect size and CI — they may show that the difference, while significant, is minor in magnitude.

Let's execute these steps:


**Important Note on Real-World Evidence:** This activity uses **synthetic data** to illustrate methodological concepts about selection bias and cross-sectional comparisons. In reality, rigorous studies using longitudinal and quasi-experimental designs (difference-in-differences, interrupted time series) have found that **PDMP must-access laws DO reduce opioid prescribing by 5-10%** when implemented (Buchmueller & Carey, 2018; Meara et al., 2016). However, the **selection bias pattern demonstrated here is real**: states with worse opioid crises were more likely to adopt strict PDMP laws, which is why cross-sectional comparisons (like we perform below) cannot establish causality. The pedagogical point—that observational policy research requires careful causal inference designs—reflects authentic challenges in this field.

**Demo:** In this demo, we examine a case where a result can be statistically significant but practically small...

In [ ]:
# 1. Filter baseline data for states without must-access PDMP
baseline = df[df['wave'] == 16].copy()
no_pdmp_group = baseline[baseline['le_l_rxmonit__addr1__pdmp__mustaccess_indicator'] == 0]

# Outcome: opioid prescribing rate
rate_var = 'le_l_rxopioid__addr1__opioidrx_rate__yb0'
rates = no_pdmp_group[rate_var].dropna()
n = len(rates)
mean_rate = rates.mean()
sd_rate = rates.std(ddof=1)
se_rate = sd_rate / np.sqrt(n)
print(f"No-PDMP states: mean rx rate = {mean_rate:.2f}, SD = {sd_rate:.2f}, n = {n}")

# 2. One-sample t-test (H0: mu = 55, Ha: mu > 55)
mu0 = 55.0
dfree = n - 1
t_stat = (mean_rate - mu0) / se_rate
p_val = stats.t.sf(t_stat, dfree)  # one-sided (greater)

# 3. 95% CI for mean
t_crit = stats.t.ppf(0.975, dfree)
ci_low = mean_rate - t_crit * se_rate
ci_high = mean_rate + t_crit * se_rate

# 4. Cohen's d
d = (mean_rate - mu0) / sd_rate

print(f"t({dfree}) = {t_stat:.3f}, one-tailed p = {p_val:.3g} (H_a: mu > 55)")
print(f"95% CI = [{ci_low:.2f}, {ci_high:.2f}] per 100")
print(f"Cohen's d = {d:.3f}")


And now a visualization: we will plot the distribution of state prescribing rates for the no-PDMP group and mark the national average (55) on it.


In [ ]:
# Plot a histogram of opioid prescribing rates in no-PDMP states
plt.figure(figsize=(6,4))
plt.hist(rates, bins=20, color='orange', edgecolor='white')
plt.axvline(mu0, color='red', linestyle='--', linewidth=2, label=f"Nat'l avg = {mu0}")
plt.axvline(mean_rate, color='black', linestyle='-', linewidth=2, label=f"Mean (no PDMP) = {mean_rate:.1f}")
plt.xlabel('Opioid prescriptions per 100 people')
plt.ylabel('Number of individuals (in sample)')
plt.title('State Opioid Prescribing Rates (No PDMP Must-Access)')
plt.legend()
savefig('03_demo_pdmp_prescribing')
plt.show()


**Observations:** The mean opioid prescribing rate in states without a mandatory PDMP is **52.73 per 100 people** (SD = 12.68, n = 9,068), which is **below** the national average of 55. The one-sample test yields **t(9067) = -17.077**, with a **one-tailed p = 1.0** for the alternative hypothesis H<sub>a</sub>: μ > 55. This result is the **opposite** of what our one-tailed hypothesis predicted! The 95% confidence interval is **[52.46, 52.99]**, which does **not** include 55 and is entirely below it. **Cohen's d = -0.179**, a small negative effect.

**What's going on?** We hypothesized that states *without* strict PDMP monitoring would have *higher* prescribing rates (thinking "no oversight = more prescriptions"). But the data show the opposite: **non-PDMP states actually prescribe LESS than the national average**. At first glance, this seems counterintuitive—shouldn't monitoring laws *reduce* prescribing?

**The Real Story - Selection Bias:** This is a classic example of **confounding by indication** in policy research. States don't adopt policies randomly:

1. **States WITH PDMP laws**: Likely adopted them *because* they had **high baseline prescribing rates** and a serious opioid problem. The policy was a **response to crisis**.
2. **States WITHOUT PDMP laws**: May have had **lower baseline prescribing** and didn't feel the policy was urgent or necessary. No crisis, no policy.

So when we compare them cross-sectionally, we're seeing the **pre-existing differences** that drove policy adoption, not the policy's effect. States with PDMP laws started from a worse position; states without them had less of a problem to begin with.

**Wrong-Tailed Test:** We tested H<sub>a</sub>: μ > 55, expecting no-PDMP states to be *above* average. But they're actually *below* average (p = 1.0 for that tail). If we had tested H<sub>a</sub>: μ < 55, we'd get p ≈ 0 and "reject" the null—but this would be **misleading** because it doesn't account for selection bias. The "significant" result wouldn't tell us whether PDMP laws work; it would just confirm that states self-select into policies based on need.

**Key Lessons:**
1. **Cross-sectional comparisons can't establish causality**: We can't compare states with/without policies as if they were randomly assigned. Pre-existing differences confound everything.
2. **Direction matters in one-tailed tests**: We picked the wrong tail based on naive reasoning. Always explore data first before committing to a directional hypothesis.
3. **Small effect, big significance**: Even though the result is highly significant (huge sample, narrow CI), the effect size is small (d = -0.179 ≈ 2.3 prescriptions per 100 people difference). Statistical significance ≠ practical importance.
4. **Policy evaluation is hard**: To know if PDMP laws *work*, we'd need longitudinal data (before/after comparisons within states) or quasi-experimental designs (difference-in-differences, instrumental variables, etc.)—not just snapshots of who has which policies.

**Bottom line:** This example shows why **observational policy research** requires careful design. A "significant" cross-sectional difference tells us almost nothing about whether a policy is effective when states self-select into policies based on the very outcomes we're trying to measure.

### Your Turn 3: The Other Side of Selection Bias - Testing PDMP States

**Context:** In Activity 3, we discovered that states **without** PDMP laws actually have **lower** prescribing rates than the national average—a surprising finding that revealed **selection bias**: states don't adopt policies randomly, but rather in response to crises.

**The Flip Side:** If no-PDMP states have low prescribing (they didn't need the policy), then logically, states that **did** adopt PDMP laws should have **high** prescribing rates—they adopted the policy *because* they had a serious opioid crisis.

**Your Task:** Test this hypothesis by analyzing states **with** must-access PDMP laws. Determine whether their prescribing rates are **above** the national average, and create visualizations comparing both policy groups.

**Steps to Complete:**

**Part 1: Statistical Analysis**
1. Filter baseline data (wave 16) for states WITH must-access PDMP laws (indicator = 1)
2. Extract opioid prescribing rate data (`le_l_rxopioid__addr1__opioidrx_rate__yb0`)
3. Set reference value (μ₀ = 55, national average)
4. Perform **one-sample t-test** with one-tailed alternative (Hₐ: μ > 55)
5. Calculate 95% confidence interval
6. Calculate Cohen's d effect size
7. Interpret results in context of selection bias

**Part 2: Visualization**
1. Create a **two-panel figure** showing:
   - **Left panel**: Histogram of PDMP state prescribing rates with reference lines
   - **Right panel**: Side-by-side comparison of No-PDMP (from Demo) vs. PDMP states (Your Turn)
2. Add arrows and annotations to highlight the selection bias pattern
3. Save figure to the figures directory

**Important Notes:**
- This is a **one-tailed test** (Hₐ: μ > 55) because we expect PDMP states to have adopted the policy in response to high prescribing
- The Demo found no-PDMP states had mean = 52.73 (BELOW average). If PDMP states have mean ABOVE average, this confirms selection bias.
- Remember: Even if PDMP states have high prescribing WITH the policy, this doesn't mean the policy failed—it means they started from a worse position (crisis → policy adoption)
- Use baseline data (wave 16) to capture pre-policy or early-policy conditions

---

### Getting Started with Copilot

**Part 1: Statistical Analysis (First Code Cell)**

Use these prompts sequentially to build your statistical analysis:

**Prompt 1:** Filter `df` to wave 16 as `baseline`. Create `pdmp_states` by filtering for must-access PDMP indicator equal to 1. Define `var` as the opioid prescribing rate variable and extract the data, dropping NaN values. Calculate sample size `n`.

**Prompt 2:** Set `mu0 = 55.0` as the national average reference value and `alternative = 'greater'` for the one-tailed test. Compute sample statistics: `mean_val`, `sd_val` (with ddof=1), and `se_val`.

**Prompt 3:** Print a formatted header "STATES WITH PDMP MUST-ACCESS LAWS" with dividers. Display the sample mean, SD, SE, n, and national average with descriptive labels.

**Prompt 4:** Perform a one-sample t-test comparing `mean_val` to `mu0`. Calculate degrees of freedom `dfree`, t-statistic `t_stat`, and one-tailed p-value `p_val` using `stats.t.sf()` (upper tail).

**Prompt 5:** Calculate the 95% confidence interval using the critical t-value from `stats.t.ppf(0.975, dfree)`. Calculate Cohen's d as `d_val = (mean_val - mu0) / sd_val`.

**Prompt 6:** Print the test results: t-statistic with df, one-tailed p-value with hypothesis notation, 95% CI, and Cohen's d. Format numbers appropriately (2-3 decimal places).

**Prompt 7:** Create an interpretation guide that prints whether PDMP states prescribe significantly more than average. If p < 0.05, explain how this supports the selection bias story (states with high prescribing adopted policies). Include effect size interpretation and critical thinking notes about why cross-sectional comparisons can't establish causality.

---

**Part 2: Visualization (Second Code Cell)**

Use these prompts to build your visualization code:

**Prompt 8:** Create a figure with 2 subplots side-by-side using `plt.subplots(1, 2, figsize=(13, 5))`. In the left panel, create a histogram of PDMP state prescribing rates (`data`) with 25 bins, dark orange color, black edges, and alpha=0.7.

**Prompt 9:** Add two vertical lines to the left panel: one for the national average (`mu0`, red dashed, linewidth=2.5) and one for the PDMP states mean (`mean_val`, dark red solid, linewidth=2.5). Add labels, title "Prescribing Rates in PDMP Must-Access States", and legend. Include grid on y-axis.

**Prompt 10:** In the right panel, create a side-by-side bar chart comparing No-PDMP states (mean = 52.73 from Demo) and PDMP states (`mean_val`). Use light blue for No-PDMP and dark orange for PDMP. Add a horizontal line for the national average (55). Label bars with their values.

**Prompt 11:** Add arrows with annotations showing direction from national average. If no-PDMP mean is below 55, add a downward arrow labeled "BELOW avg" in blue. If PDMP mean is above 55, add an upward arrow labeled "ABOVE avg" in dark orange. Use `ax2.annotate()` with arrowprops.

**Prompt 12:** Format the right panel with appropriate axis labels, title "Selection Bias in Policy Adoption: PDMP vs. Non-PDMP States", legend, and grid. Add a figure suptitle "Your Turn 3: The Other Side of Selection Bias". Save with `savefig('yt3_pdmp_selection_bias')` and display with `plt.show()`.


In [ ]:
=== Your Turn 3: Statistical Analysis - PDMP States ===

# TODO: Filter data to baseline (wave 16) and get PDMP states
# baseline = df[df['wave'] == 16].copy()
# pdmp_states = baseline[baseline['le_l_rxmonit__addr1__pdmp__mustaccess_indicator'] == 1]
# var = 'le_l_rxopioid__addr1__opioidrx_rate__yb0'
# data = ..., n = ...

# TODO: Set reference value and alternative hypothesis
# mu0 = 55.0 (national average)
# alternative = 'greater' (one-tailed: Ha: mu > 55)

# TODO: Compute sample statistics
# mean_val, sd_val (ddof=1), se_val

# TODO: Print formatted summary statistics
# Header with "STATES WITH PDMP MUST-ACCESS LAWS"
# Display mean, SD, SE, n, reference value

# TODO: Perform one-sample t-test (one-tailed upper)
# dfree, t_stat, p_val using stats.t.sf()

# TODO: Calculate 95% confidence interval
# t_crit from stats.t.ppf(0.975, dfree)
# ci_low, ci_high

# TODO: Calculate Cohen's d effect size
# d_val = (mean_val - mu0) / sd_val

# TODO: Print test results
# t-statistic with df, one-tailed p-value, 95% CI, Cohen's d

# TODO: Print interpretation guide
# If p < 0.05: explain selection bias pattern
# Effect size interpretation (small/medium/large)
# Critical thinking notes about causality

# your code here


In [ ]:
=== Your Turn 3: Visualization - PDMP Selection Bias ===

# TODO: Create figure with 2 subplots (1 row, 2 columns)
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# TODO: LEFT PANEL - Histogram of PDMP state prescribing rates
# ax1.hist(data, bins=25, color='darkorange', edgecolor='black', alpha=0.7)
# Add vertical lines for national avg (mu0, red dashed) and PDMP mean (mean_val, dark red solid)
# Labels, title, legend, grid

# TODO: RIGHT PANEL - Side-by-side comparison bar chart
# no_pdmp_mean = 52.73 (from Activity 3 Demo)
# Create bar chart comparing no-PDMP and PDMP states
# x_pos = [1, 2], means = [no_pdmp_mean, mean_val]
# colors = ['lightblue', 'darkorange']
# Add horizontal line for national avg (55)

# TODO: Add value labels on bars
# Use ax2.text() to display mean values above bars

# TODO: Add arrows showing direction from national average
# If no_pdmp_mean < mu0: downward arrow, label "BELOW avg" (blue)
# If pdmp_mean > mu0: upward arrow, label "ABOVE avg" (dark orange)
# Use ax2.annotate() with arrowprops

# TODO: Format right panel
# Axis labels, title "Selection Bias in Policy Adoption: PDMP vs. Non-PDMP States"
# Legend, grid, appropriate axis limits

# TODO: Add figure suptitle and save
# plt.suptitle('Your Turn 3: The Other Side of Selection Bias', ...)
# savefig('yt3_pdmp_selection_bias')
# plt.show()

# TODO: Print confirmation message

# your code here


**Your Interpretation:**

After running the analysis above, answer the following questions:

1. **Descriptive Results:**
   - What is the mean prescribing rate in PDMP states?
   - Is it above or below the national average (55)?
   - What is the raw difference from the national average?

2. **Statistical Significance:**
   - Is the difference statistically significant? (Check the p-value)
   - Does the 95% CI include 55? What does this tell you?
   - What is Cohen's d? Is the effect size small, medium, or large?

3. **The Selection Bias Story:**
   - Combining Activity 3 Demo (no-PDMP states had LOW prescribing) and Your Turn 3 (PDMP states had ??? prescribing), what pattern emerges?
   - Does this support the hypothesis that states adopt policies **in response to crises** rather than randomly?
   - What does this tell us about the limitations of cross-sectional policy comparisons?

4. **Causal Inference Challenge:**
   - If PDMP states have higher prescribing rates even WITH the policy in place, does that mean the policy doesn't work?
   - Why can't we answer "Do PDMP laws reduce prescribing?" with this cross-sectional design?
   - What research design would you need to properly evaluate whether PDMP laws are effective? (Hint: think about longitudinal/before-after comparisons, or matching states with similar baseline characteristics)

5. **Practical vs. Statistical Significance:**
   - Even if the result is statistically significant, how large is the practical difference in prescribing rates?
   - From a public health perspective, is the magnitude meaningful?
   - How does sample size affect your ability to detect even small differences?

Write your answers in a markdown cell below, citing specific statistics from your output.